# Load-Dependent Multi-Server FCFS Queue

This example demonstrates load-dependent modeling with multi-server FCFS scheduling.

## Key Features:
- **Load dependence**: Service capacity scales with queue population
- **Multi-server**: Up to c=2 servers available
- **FCFS scheduling**: First-Come-First-Served discipline
- **Scaling function**: min(queue_population, c) active servers

## Comparison:
1. **Standard multi-server**: Fixed number of servers
2. **Load-dependent**: Dynamic server allocation based on queue length

In [ ]:
// Kotlin notebook
import jline.*
import jline.lang.*
import jline.lang.nodes.*
import jline.lang.processes.*
import jline.lang.constant.*
import jline.solvers.mva.*
import jline.solvers.nc.*
import jline.solvers.ctmc.*
import jline.util.matrix.*

In [ ]:
val N = 16 // number of jobs
val c = 2  // number of servers

// Standard multi-server model
val model = Network("model")
val nodes = arrayOfNulls<Node>(2)
nodes[0] = Delay(model, "Delay")
nodes[1] = Queue(model, "Queue1", SchedStrategy.FCFS)

val jobclass = ClosedClass(model, "Class1", N, nodes[0], 0)

nodes[0]!!.setService(jobclass, Exp.fitMean(1.0)) // mean = 1
nodes[1]!!.setService(jobclass, Exp.fitMean(1.5)) // mean = 1.5
(nodes[1] as Queue).numberOfServers = c

model.link(Network.serialRouting(nodes.toList()))

val msT = NC(model).avgTable
println("Standard Multi-Server Results:")
msT.print()

In [ ]:
// Load-dependent model with scaling function that depends on the total queue
val ldmodel = Network("ldmodel")
val ldnodes = arrayOfNulls<Node>(2)
ldnodes[0] = Delay(ldmodel, "Delay")
ldnodes[1] = Queue(ldmodel, "Queue1", SchedStrategy.FCFS)

val ldjobclass = ClosedClass(ldmodel, "Class1", N, ldnodes[0], 0)

ldnodes[0]!!.setService(ldjobclass, Exp.fitMean(1.0)) // mean = 1
ldnodes[1]!!.setService(ldjobclass, Exp.fitMean(1.5)) // mean = 1.5

// Set load dependence: min(population, c) - multi-server with c servers
val ldFunction = DoubleArray(N + 1) { i ->
    kotlin.math.min(kotlin.math.max(i, 1).toDouble(), c.toDouble())
}
ldnodes[1]!!.setLoadDependence(ldFunction.toList())

ldmodel.link(Network.serialRouting(ldnodes.toList()))

println("\nLoad-dependent function: ${ldFunction.toList()}")

In [ ]:
// Solve with CTMC
println("\n=== Load-Dependent CTMC Results ===")
try {
    val lldAvgTableCTMC = CTMC(ldmodel).avgTable
    lldAvgTableCTMC.print()
} catch (e: Exception) {
    println("CTMC solver failed: ${e.message}")
}

In [ ]:
// Try NC solver with different methods
println("\n=== NC Solver Results ===")
try {
    val lldAvgTableNC = NC(ldmodel).avgTable
    println("NC default:")
    lldAvgTableNC.print()
} catch (e: Exception) {
    println("NC default failed: ${e.message}")
}

// Try alternative NC methods
val ncMethods = listOf("rd", "nrp", "nrl")
for (method in ncMethods) {
    try {
        val solver = NC(ldmodel, "method", method)
        val result = solver.avgTable
        println("\nNC $method:")
        result.print()
    } catch (e: Exception) {
        println("NC $method failed: ${e.message}")
    }
}

In [ ]:
// Try MVA solvers
println("\n=== MVA Solver Results ===")
val mvaMethods = listOf("exact", "qd")
for (method in mvaMethods) {
    try {
        val solver = MVA(ldmodel, "method", method)
        val result = solver.avgTable
        println("\nMVA $method:")
        result.print()
    } catch (e: Exception) {
        println("MVA $method failed: ${e.message}")
    }
}